# 朴素贝叶斯迷你项目
想象你的老板从一个信号很差的地方给你留了条消息。有几个词根本听不清。你想基于他以前给你留的一些消息的录音，填补这些剩下的词。为此，给定一些额外的消息信息，我们将使用贝叶斯公式来计算给定单词正好能填入留白处的概率。  
回想贝叶斯公式：  
$$P(A|B)=P(B|A)*P(A)/P(B)$$
或者在我们这个案例中：  
$$P(某个特定词|前后文的词)=P(前后文的词|某个特定的词)*P(某个特定的词)/P(前后文的词)$$

### 最大可能性假设

在这个练习中我们要根据前面一个单词，找出哪个单词最有可能跟在它后面  

实现 NextWordProbability 使得你可以传入一段话，一个词，返回一个字典。这个字典的键（keys）是出现在这个词后面的词，每个键（key）的值（value）是跟在后面这个词出现的次数。  

你可以用 .split() 方法来把 sample_memo 这段话中的词用空格分割开来。

In [3]:
sample_memo = '''
Milt, we're gonna need to go ahead and move you downstairs into storage B. We have some new people coming in, and we need all the space we can get. So if you could just go ahead and pack up your stuff and move it down there, that would be terrific, OK?
Oh, and remember: next Friday... is Hawaiian shirt day. So, you know, if you want to, go ahead and wear a Hawaiian shirt and jeans.
Oh, oh, and I almost forgot. Ahh, I'm also gonna need you to go ahead and come in on Sunday, too...
Hello Peter, whats happening? Ummm, I'm gonna need you to go ahead and come in tomorrow. So if you could be here around 9 that would be great, mmmk... oh oh! and I almost forgot ahh, I'm also gonna need you to go ahead and come in on Sunday too, kay. We ahh lost some people this week and ah, we sorta need to play catch up.
'''
#
#   Maximum Likelihood Hypothesis
#
#
#   In this quiz we will find the maximum likelihood word based on the preceding word
#
#   Fill in the NextWordProbability procedure so that it takes in sample text and a word,
#   and returns a dictionary with keys the set of words that come after, whose values are
#   the number of times the key comes after that word.
#   
#   Just use .split() to split the sample_memo text into words separated by spaces.

def NextWordProbability(sampletext,word):
    sample_words=sampletext.split()
    times=CountWordTimes(sample_words,word)
    dic={}
    for index in range(len(sample_words)):
        if sample_words[index]==word and index < len(sample_words):
            key=sample_words[index+1]
            if key in dic.keys():
                dic[key]+=1
            else:
                dic[key]=1
    final={}
    for i in dic.keys():
        final[i]=float(dic[i])/times
    return final
    
def CountWordTimes(sample,word):
    time=0
    for item in sample:
        if item == word:
            time += 1
    return time

In [4]:
# 例子
NextWordProbability(sample_memo,'need')

{'all': 0.16666666666666666, 'to': 0.3333333333333333, 'you': 0.5}